In [ ]:
%pip install 'qbraid[quera,cirq,visualization]>=0.8.4' -q

In [ ]:
import cirq

qubits = [cirq.LineQubit(i) for i in range(5)]

circuit = cirq.Circuit()

circuit.append(cirq.H(qubits[0]))

for qubit in qubits[1:]:
    circuit.append(cirq.CNOT(qubits[0], qubit))

print(circuit)

In [ ]:
from qbraid import QbraidProvider

provider = QbraidProvider(api_key="YOUR_API_KEY")
provider.save_config()

In [ ]:
device = provider.get_device("quera_qasm_simulator")

print(device.status())

In [ ]:
device.metadata()

In [ ]:
job = device.run(circuit, shots=100, tags={"batch": "test_1"})

print(job.id)

In [ ]:
import time

status_final = False

while not status_final:
    status = job.status()
    print(status)
    status_final = job.is_terminal_state()
    time.sleep(3)

In [ ]:
result = job.result()

In [ ]:
result.success

In [ ]:
result.details["timeStamps"]  # executionDuration in milliseconds

In [ ]:
result.details["cost"]  # cost in qBraid credits (free)

In [ ]:
result.details["tags"]

In [ ]:
print(result.details["metadata"]["openQasm"])

In [ ]:
counts_bin = result.data.get_counts()
probs_bin = result.data.get_probabilities()
probs_dec = result.data.get_probabilities(decimal=True)
shots = result.data.to_dict()["shots"]

print(f"shots: {shots}")

print(f"probabilities:")
probs_bin

In [ ]:
from qbraid.visualization import plot_histogram, plot_distribution, animate_qpu_state

counts_dec = result.data.get_counts(decimal=True)

plot_histogram(counts_dec)

In [ ]:
plot_distribution(counts_dec)

In [ ]:
logs = result.data.get_logs()

logs

In [ ]:
qpu_state = result.data.get_qpu_state()

type(qpu_state)

In [ ]:
%matplotlib inline

animate_qpu_state(qpu_state)

In [ ]:
quera_sim_program_spec = device.profile.program_spec

quera_sim_program_spec

In [ ]:
from qbraid import ConversionGraph, QPROGRAM_REGISTRY

graph = ConversionGraph()

print(
    f"Based on your currently installed packages, you can submit to the Quera QASM simulator from the following program types:\n"
)

for program_type in QPROGRAM_REGISTRY:
    if graph.has_path(program_type, quera_sim_program_spec.alias):
        print(f"'{program_type}': {QPROGRAM_REGISTRY[program_type]}")


print(
    "\n\nNote: Only a limited subset of features within these program types will be supported, specifically \nthose that can be directly converted to an OpenQASM 2 format compatible with the QuEra simulator."
)